# Set up

In [1]:
import torch
import gpytorch
import pandas as pd
import numpy as np
import tqdm as tqdm
from linear_operator import settings

from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import pyro.distributions as dist
from pyro.infer import MCMC, NUTS
import arviz as az
import seaborn as sns

In [2]:
import GP_functions.Loss_function as Loss_function
import GP_functions.bound as bound
import GP_functions.Estimation as Estimation
import GP_functions.Training as Training
import GP_functions.Prediction as Prediction
import GP_functions.GP_models as GP_models
import GP_functions.Tools as Tools
import GP_functions.FeatureE as FeatureE

# Data

In [3]:
X_train = pd.read_csv('Data/X_train.csv', header=None, delimiter=',').values
X_test = pd.read_csv('Data/X_test.csv', header=None, delimiter=',').values

Y_train_8 = pd.read_csv('Data/Y_train_8.csv', header=None, delimiter=',').values
Y_test_8 = pd.read_csv('Data/Y_test_8.csv', header=None, delimiter=',').values

Y_train_20 = pd.read_csv('Data/Y_train_20.csv', header=None, delimiter=',').values
Y_test_20 = pd.read_csv('Data/Y_test_20.csv', header=None, delimiter=',').values

Y_train_std = pd.read_csv('Data/Y_train_std.csv', header=None, delimiter=',').values
Y_test_std = pd.read_csv('Data/Y_test_std.csv', header=None, delimiter=',').values

In [4]:
train_x = torch.tensor(X_train, dtype=torch.float32)
test_x = torch.tensor(X_test, dtype=torch.float32)

train_y_8 = torch.tensor(Y_train_8, dtype=torch.float32)
test_y_8 = torch.tensor(Y_test_8, dtype=torch.float32)

train_y_20 = torch.tensor(Y_train_20, dtype=torch.float32)
test_y_20 = torch.tensor(Y_test_20, dtype=torch.float32)

train_y = torch.tensor(Y_train_std, dtype=torch.float32)
test_y = torch.tensor(Y_test_std, dtype=torch.float32)

# Emulators

In [5]:
Device = 'cuda'

In [6]:
row_idx = 0

input_point = test_y[row_idx,:]
local_train_x, local_train_y = Tools.find_k_nearest_neighbors_GPU(input_point, train_x, train_y, k = 100)


## Training part

In [8]:
MultitaskGP_models, MultitaskGP_likelihoods = Training.train_one_row_MultitaskGP(local_train_x, local_train_y, n_tasks = train_y.shape[1], 
                                                                                     covar_type = 'RQ', lr=0.05, num_iterations=5000, patience=10, device=Device)


 21%|██        | 1051/5000 [00:24<01:32, 42.53it/s, loss=-1.85] 


In [7]:
MultitaskGP_models_lcm, MultitaskGP_likelihoods_lcm = Training.train_one_row_MultitaskGP_lcm(local_train_x, local_train_y, n_tasks = train_y.shape[1], 
                                                                                                 lr=0.05, num_iterations=5000, patience=10, device=Device)

  4%|▍         | 206/5000 [00:21<11:51,  6.74it/s, loss=-1.67]  d:\anaconda3\envs\FGPyT\lib\site-packages\linear_operator\utils\linear_cg.py:338: NumericalWarning: CG terminated in 1000 iterations with average residual norm 1.8011454343795776 which is larger than the tolerance of 1 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.
  warnings.warn(
  4%|▍         | 210/5000 [00:24<09:18,  8.57it/s, loss=-1.68] 


In [ ]:
with settings.max_cg_iterations(2000):
    MultitaskGP_models_lcm, MultitaskGP_likelihoods_lcm = Training.train_one_row_MultitaskGP_lcm(local_train_x, local_train_y, n_tasks = train_y.shape[1], 
                                                                                                 lr=0.05, num_iterations=5000, patience=10, device=Device)

## Pre

In [ ]:
full_test_preds = Prediction.preds_for_one_model(MultitaskGP_models, MultitaskGP_likelihoods, test_x[row_idx,:].unsqueeze(0).to(Device)).squeeze().detach().numpy()

## Estimation

### Point estimation

In [ ]:
bounds = bound.get_bounds(local_train_x)

estimated_params, func_loss = Estimation.multi_start_estimation(MultitaskGP_models, MultitaskGP_likelihoods, row_idx, test_y, bounds, Estimation.estimate_params_for_one_model_Adam, 
                                                                num_starts=5, num_iterations=2000, lr=0.01, patience=10, 
                                                                attraction_threshold=0.1, repulsion_strength=0.1, device=Device)


# full_estimated_params = estimated_params.detach().numpy()


In [ ]:
estimated_params

### MCMC 

In [ ]:
mcmc_result_Normal = Estimation.run_mcmc_Normal(Prediction.preds_for_one_model, MultitaskGP_models, MultitaskGP_likelihoods, row_idx, test_y, local_train_x, num_sampling=1000, warmup_step=500)

posterior_samples_Normal = mcmc_result_Normal.get_samples()

param_names = [f'param_{i}' for i in range(len(bounds))]

posterior_means_array = np.zeros(len(param_names))


for idx, param_name in enumerate(param_names):
    samples = posterior_samples_Normal[param_name]
    if samples.ndim > 1:
        samples = samples.reshape(-1)
    mean_value = torch.mean(samples).item()
    posterior_means_array[idx] = mean_value